In [ ]:
# This notebook runs regression analysis (Poisson, NB, Quasi-poisson) and robustness checks on the combined violence dataset. 

In [ ]:
import pandas as pd
from patsy import dmatrices
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_pickle('../combined datasets/dataset_for_regression.pkl')

In [ ]:
df.info()

In [ ]:
df = df.drop(columns = ['total_cases_weibo', 'total_protest_num', 'gdp', 'num_of_hired_vio_in_protest', 'total_violence_cases', 'hired_cases', 'hired_ratio'])

In [ ]:
df['all_hired_violence'].sum()

In [ ]:
df.describe()

# Fit Poisson Regressions

In [ ]:
def fit_poisson_regression(X, y):
    """
    Fit a poisson regression model using independent variables (X) and the dependent variable (y).

    Parameters:
    X: IVs
    y: DV

    Returns:
    results: fitted model results.
    """

    # Add a constant as intercept
    X = sm.add_constant(X)

    # Define the Poisson model
    poisson_model = sm.GLM(y, X, family = sm.families.Poisson())

    # Fit the model
    results = poisson_model.fit()

    # Print the summary of the fitted model
    print(results.summary())
    
    return results

In [ ]:
# Define dependent variable
y = df['all_hired_violence']

# Define independent variables
control_var = ['cpi', 'unemp_rate', 'urbanpop_by_totalpop', 'migpop_by_totalpop', 'gdp_pc']
market_var = ['landsale_by_govrev', 'median_land_size']
gov_var = ['median_corrup_cases', 'median_audit_cases', 'median_protest_num']
statcap_var = ['securityexp_pc', 'govexp_by_gdp', 'govrev_by_gdp']

X0 = df[control_var]
X_mar =  df[control_var + market_var]
X_gov = df[control_var + gov_var]
X_statcap = df[control_var + statcap_var]
X_full = df[control_var + market_var + gov_var + statcap_var]

# Fit models
baseline_poisson = fit_poisson_regression(X0, y)
market_poisson = fit_poisson_regression(X_mar, y)
governance_poisson = fit_poisson_regression(X_gov, y)
statcap_poisson = fit_poisson_regression(X_statcap, y)
full_poisson = fit_poisson_regression(X_full, y)

# PCA

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def add_pca_component(df, columns, n_components=1, component_name="pca_component"):
    """
    Performs PCA on specified columns of a DataFrame and adds the first principal component as a new column.

    Parameters:
    - df (DataFrame): The input DataFrame containing the data.
    - columns (list): List of columns to perform PCA on.
    - n_components (int): Number of principal components to retain (default is 1).
    - component_name (str): Name for the new column with the PCA result.

    Returns:
    - DataFrame: The original DataFrame with an additional column for the first principal component.
  
    """
    # Step 1: Standardize the data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df[columns])

    # Step 2: Perform PCA
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X_scaled)

    # Get the loadings (principal component coefficients for each original feature)
    loadings = pd.DataFrame(pca.components_.T, columns=[f'PC{i+1}' for i in range(n_components)], index=columns)

    # Add the first principal component to the DataFrame
    df[component_name] = X_reduced[:, 0] if n_components == 1 else X_reduced

    return df


In [ ]:
df = add_pca_component(df, market_var, n_components = 1, component_name = 'market_pca')
df = add_pca_component(df, gov_var, n_components = 1, component_name = 'gov_pca')
df = add_pca_component(df, statcap_var, n_components = 1, component_name = 'statcap_pca')

In [ ]:
pca_var = ['market_pca', 'gov_pca', 'statcap_pca']

X_pca = df[control_var + pca_var]

pca_poisson = fit_poisson_regression(X_pca, y)

# Overdispersion Checks for Poisson Models

In [ ]:
def check_overdispersion(model):
    """
    Checks for overdispersion in a Poisson regression model using deviance/df.

    Parameters:
    - model: A fitted Poisson regression model from statsmodels.

    Returns:
    - Dispersion statistic.
    """
    # Deviance
    deviance = model.deviance

    # Degrees of freedom
    df = model.df_resid

    # Calculate dispersion
    dispersion = deviance/df
    
    return dispersion


In [ ]:
all_models = [baseline_poisson, market_poisson, governance_poisson, statcap_poisson, full_poisson, pca_poisson]

for model in all_models:
    print(check_overdispersion(model))

In [ ]:
# full model - goodness of fit test
from scipy.stats import chi2

# Get the expected counts
expected_counts = full_poisson.fittedvalues

# Compute the Pearson Chi-Square statistic
chi_square_stat = np.sum((y - expected_counts) ** 2 / expected_counts)

# Compute the degrees of freedom
n_observations = len(y)
n_parameters = len(full_poisson.params)
degrees_of_freedom = n_observations - n_parameters

# Compute the p-value
p_value = 1 - chi2.cdf(chi_square_stat, degrees_of_freedom)

# Print results
print("Pearson Chi-Square Statistic:", chi_square_stat)
print("Degrees of Freedom:", degrees_of_freedom)
print("P-value:", p_value)

if p_value < 0.05:
    print("The model does not fit the data well (reject null hypothesis).")
else:
    print("The model fits the data well (fail to reject null hypothesis).")


In [ ]:
# note that chi-square statistics is larger than the degrees of freedom, indicating overdispersion

In [ ]:
# plot residuals

import matplotlib.pyplot as plt

plt.scatter(full_poisson.fittedvalues, full_poisson.resid_pearson)

# Add horizontal reference lines
for h in [-3, -2, 0, 2, 3]:
    plt.axhline(y=h, color='red', linestyle='dotted')
    
plt.xlabel('Fitted Values')
plt.ylabel('Pearson Residuals')
plt.title('Residuals vs Fitted Values')
plt.show()


In [ ]:
# Combine data into a DataFrame for inspection
results_df = pd.DataFrame({
    'province': df['province'],
    'Observed Values': y,
    'Fitted Values': full_poisson.fittedvalues,
    'Standardized Residuals': full_poisson.resid_pearson})

# Define outlier threshold
outlier_threshold = 2

# Filter rows with standardized residuals above or below the threshold
outliers = results_df[(results_df['Standardized Residuals'] > outlier_threshold) |
                      (results_df['Standardized Residuals'] < -outlier_threshold)]

# Display the rows corresponding to outliers
print(outliers)



In [ ]:
# plot residuals

import matplotlib.pyplot as plt

plt.scatter(full_poisson_new.fittedvalues, full_poisson_new.resid_pearson)

# Add horizontal reference lines
for h in [-3, -2, 0, 2, 3]:
    plt.axhline(y=h, color='red', linestyle='dotted')
    
plt.xlabel('Fitted Values')
plt.ylabel('Pearson Residuals')
plt.title('Residuals vs Fitted Values')
plt.show()

# Drop outlier and rerun poisson

In [ ]:
df_drop_outlier = df[df['province'] != 'Hebei']

In [ ]:
# Define dependent variable
y_new = df_drop_outlier['all_hired_violence']

X0_new = df_drop_outlier[control_var]
X_mar_new =  df_drop_outlier[control_var + market_var]
X_gov_new = df_drop_outlier[control_var + gov_var]
X_statcap_new = df_drop_outlier[control_var + statcap_var]
X_full_new = df_drop_outlier[control_var + market_var + gov_var + statcap_var]

# Fit models
baseline_poisson_new = fit_poisson_regression(X0_new, y_new)
market_poisson_new = fit_poisson_regression(X_mar_new, y_new)
governance_poisson_new = fit_poisson_regression(X_gov_new, y_new)
statcap_poisson_new = fit_poisson_regression(X_statcap_new, y_new)
full_poisson_new = fit_poisson_regression(X_full_new, y_new)

In [ ]:
# check overdispersion
all_models_new = [baseline_poisson_new, market_poisson_new, governance_poisson_new, statcap_poisson_new, full_poisson_new]

for model in all_models_new:
    print(check_overdispersion(model))

In [ ]:
# Combine data into a DataFrame for inspection
results_df = pd.DataFrame({
    'province': df_drop_outlier['province'],
    'Observed Values': y,
    'Fitted Values': full_poisson_new.fittedvalues,
    'Standardized Residuals': full_poisson_new.resid_pearson})

# Define outlier threshold
outlier_threshold = 2

# Filter rows with standardized residuals above or below the threshold
outliers = results_df[(results_df['Standardized Residuals'] > outlier_threshold) |
                      (results_df['Standardized Residuals'] < -outlier_threshold)]

# Display the rows corresponding to outliers
print(outliers)



# Fit Negative Binomial Models

In [ ]:
# reference: https://timeseriesreasoning.com/contents/negative-binomial-regression-model/
# https://python.plainenglish.io/a-step-by-step-guide-to-count-data-analysis-in-python-a981544fc4f0

In [ ]:
def fit_negative_binomial(X, y, alpha):
    """
    Fit a negative binomial model using independent variables (X) and the dependent variable (y).

    Parameters:
    X: IVs
    y: DV

    Returns:
    results: fitted model results.
    """

    # Define the NB2 model, using default value for alpha
    nb_model = sm.GLM(y, X, family = sm.families.NegativeBinomial(alpha=alpha))

    # Fit the model
    results = nb_model.fit()

    # Print the summary of the fitted model
    print(results.summary())
    
    return results

In [ ]:
from sklearn.preprocessing import StandardScaler

X0_scaled = StandardScaler().fit_transform(X0)
X_mar_scaled = StandardScaler().fit_transform(X_mar)
X_gov_scaled = StandardScaler().fit_transform(X_gov)
X_statcap_scaled = StandardScaler().fit_transform(X_statcap)
X_full_scaled = StandardScaler().fit_transform(X_full)
X_pca_scaled = StandardScaler().fit_transform(X_pca)

In [ ]:
import numpy as np

# Check for NaN or infinite values in `y`
print("y contains NaN or Inf:", np.any(np.isnan(y)) or np.any(np.isinf(y)))

# Check for NaN or infinite values in `X_full`
print("X_gov_scaled contains NaN or Inf:", np.any(np.isnan(X_full_scaled)) or np.any(np.isinf(X_full_scaled)))


In [ ]:
# Fit models
baseline_nb = fit_negative_binomial(X0_scaled, y, 1)
market_nb = fit_negative_binomial(X_mar_scaled, y, 1)
governance_nb = fit_negative_binomial(X_gov_scaled, y, 0.5)
statcap_nb = fit_negative_binomial(X_statcap_scaled, y, 1)
full_nb = fit_negative_binomial(X_full_scaled, y, 0.5)
pca_nb = fit_negative_binomial(X_pca_scaled, y, 1)

# Negative Binomial Model Evaluation

In [ ]:
# log likelihood

all_nb_models = [baseline_nb, market_nb, governance_nb, statcap_nb, full_nb, pca_nb]

for model_name in all_nb_models:
    print(f'loglikelihood: {model_name.llf}')


In [ ]:
# AIC BIC

for model in all_nb_models:
    print(f"AIC: {model.aic}")
    print(f"BIC: {model.bic}")


In [ ]:
# Pearson Chi-Square Goodness-of-Fit Test

from scipy.stats import chi2

def get_pearson_chi(model):
    
    # Calculate observed and predicted values
    observed = y
    predicted = model.fittedvalues
    
    # Compute Pearson residuals
    pearson_residuals = (observed - predicted) / np.sqrt(predicted)
    
    # Calculate Pearson Chi-Square statistic
    pearson_chi2 = np.sum(pearson_residuals**2)
    
    # Degrees of freedom
    df = model.df_resid
    
    # Calculate the p-value
    p_value = 1 - chi2.cdf(pearson_chi2, df)
    
    # Print results
    print(f"Pearson Chi-Square Statistic: {pearson_chi2}")
    print(f"Degrees of Freedom: {df}")
    print(f"p-value: {p_value}")
    
    # Interpret the fit
    if p_value > 0.05:
        print("The model fits the data well (fail to reject null hypothesis).")
    else:
        print("The model does not fit the data well (reject null hypothesis).")

for model in all_nb_models:
    print(get_pearson_chi(model))


In [ ]:
# Calculate McFadden's pseudo-R²

for model in all_nb_models:
    null_llf = model.llnull  # log-likelihood of the null model
    mcfadden_r2 = 1 - (model.llf / null_llf)
    print(f"McFadden's R²: {mcfadden_r2}")


In [ ]:
# In essence, a negative pseudo R square indicates that your model does not explain the data as well as the null model.

In [ ]:
# plot residuals

import matplotlib.pyplot as plt

plt.scatter(full_nb.fittedvalues, full_nb.resid_pearson)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Fitted Values')
plt.ylabel('Pearson Residuals')
plt.title('Residuals vs Fitted Values')
plt.show()


In [ ]:
# plot y vs fitted values

import matplotlib.pyplot as plt

plt.scatter(full_nb.fittedvalues, y)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Fitted Values')
plt.ylabel('True values')
plt.title('True vs Fitted Values')
plt.show()


# Quasi-poisson

In [ ]:
# Calculate scale (dispersion) factor
scale_factor = full_poisson.deviance / full_poisson.df_resid

# Refit the model with adjusted standard errors
quasi_poisson_results = full_poisson.get_robustcov_results(cov_type='HC0')
print(quasi_poisson_results.summary())


In [ ]:
def fit_quasi_poisson_regression(X, y):
    """
    Fit a poisson regression model using independent variables (X) and the dependent variable (y).

    Parameters:
    X: IVs
    y: DV

    Returns:
    results: fitted model results.
    """

    # Add a constant as intercept
    X = sm.add_constant(X)

    # Define the Poisson model
    poisson_model = sm.GLM(y, X, family = sm.families.Poisson())

    # Fit the model
    results = poisson_model.fit(cov_type='HC0')

    # Print the summary of the fitted model
    print(results.summary())
    
    return results

In [ ]:
# Fit models
baseline_q_poisson = fit_quasi_poisson_regression(X0, y)
market_q_poisson = fit_quasi_poisson_regression(X_mar, y)
governance_q_poisson = fit_quasi_poisson_regression(X_gov, y)
statcap_q_poisson = fit_quasi_poisson_regression(X_statcap, y)
full_q_poisson = fit_quasi_poisson_regression(X_full, y)

In [ ]:
all_qp_models = [baseline_q_poisson, market_q_poisson, governance_q_poisson, statcap_q_poisson, full_q_poisson]

for model in all_qp_models:
    print(check_overdispersion(model))

# VIF

In [ ]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

def get_vif(x):
    vif = pd.DataFrame()
    x = add_constant(x)
    vif['variables'] = x.columns
    vif['vif'] = [variance_inflation_factor(x.values, i) for i in range(x.shape[1])]
    return vif

for X in [X0, X_mar, X_gov, X_statcap, X_full]:
    print(get_vif(X))



In [ ]:
# Define dependent variable
y = df['all_hired_violence']

# Revise independent variables

# drop 'urbanpop_by_totalpop' from control variables
control_var_new = ['cpi', 'unemp_rate', 'migpop_by_totalpop'] # first two vars reflect overall economic conditions

# drop 'landsale_by_govrev'OR 'median_land_size'???  
# df['gdp_land_interaction'] = df['gdp_pc'] * df['median_land_size']
# df['gdp_landrev_interaction'] = df['gdp_pc'] * df['landsale_by_govrev']
df['land_size_rev'] = df['median_land_size'] * df['landsale_by_govrev']
# df['land_size_square'] = df['median_land_size'] ** 2
market_var_new = ['gdp_pc', 'median_land_size', 'landsale_by_govrev'] # market variables reflect level of urbanization

# df['audit_corrup_interaction'] = df['median_corrup_cases'] * df['median_audit_cases']
gov_var_new = ['median_corrup_cases', 'median_audit_cases'] # drop 'median_corrup_cases' OR 'median_audit_cases'??? 

df['log_govrev'] = np.log(df['govrev_by_gdp'])
df['sqrt_govrev'] = np.sqrt(df['govrev_by_gdp'])
df['reciprocal_govrev'] = 1 / df['govrev_by_gdp']
statcap_var_new = ['securityexp_pc', 'govrev_by_gdp'] # drop 'govexp_by_gdp' OR 'govrev_by_gdp'because of collinearity

X0_new = df[control_var_new]
X_mar_new =  df[control_var_new + market_var_new]
X_gov_new = df[control_var_new + gov_var_new]
X_statcap_new = df[control_var_new + statcap_var_new]

control_reduced = ['unemp_rate', 'migpop_by_totalpop']
market_reduced = ['gdp_pc']
gov_reduced = ['median_corrup_cases']
statcap_reduced = ['securityexp_pc']

X_full_new = df[control_reduced + market_reduced + gov_reduced + statcap_reduced]

# Fit models
baseline_qp_new = fit_quasi_poisson_regression(X0_new, y)
market_qp_new = fit_quasi_poisson_regression(X_mar_new, y)
governance_qp_new = fit_quasi_poisson_regression(X_gov_new, y)
statcap_qp_new = fit_quasi_poisson_regression(X_statcap_new, y)
full_qp_new = fit_quasi_poisson_regression(X_full_new, y)

In [ ]:
for X in [X0_new, X_mar_new, X_gov_new, X_statcap_new, X_full_new]:
    print(get_vif(X))


In [ ]:
all_new_models = [baseline_qp_new, market_qp_new, governance_qp_new, statcap_qp_new, full_qp_new]

for model in all_new_models:
    # Pseudo log-likelihood (based on Poisson assumption)
    log_likelihood = model.llf
    
    # Calculate Pseudo R^2 (McFadden)
    ll_null = sm.GLM(y, sm.add_constant(np.ones_like(y)), family=sm.families.Poisson()).fit().llf
    pseudo_r2_mcfadden = 1 - log_likelihood / ll_null
    
    print("Log-Likelihood:", log_likelihood)
    print("Pseudo R^2 (McFadden):", pseudo_r2_mcfadden)

In [ ]:
# export modeling results 

# Extract coefficients and p-values
summary_model1 = baseline_qp_new.summary2().tables[1]
summary_model2 = market_qp_new.summary2().tables[1]
summary_model3 = governance_qp_new.summary2().tables[1]
summary_model4 = statcap_qp_new.summary2().tables[1]
summary_model5 = full_qp_new.summary2().tables[1]

# Function to apply stars based on p-value thresholds
def add_significance_stars(df):
    def apply_stars(row):
        coef = row['Coef.']
        p_value = row['P>|z|']
        # Apply stars based on p-value
        if p_value < 0.001:
            return f"{coef:.3f}***"
        elif p_value < 0.01:
            return f"{coef:.3f}**"
        elif p_value < 0.051:
            return f"{coef:.3f}*"
        else:
            return f"{coef:.3f}"
    
    df['Coef. (with stars)'] = df.apply(apply_stars, axis=1)
    return df

# Keep only the coefficients and p-values for each model and apply stars
coefficients1 = add_significance_stars(summary_model1[['Coef.', 'P>|z|']]).reset_index(drop=True)
coefficients2 = add_significance_stars(summary_model2[['Coef.', 'P>|z|']]).reset_index(drop=True)
coefficients3 = add_significance_stars(summary_model3[['Coef.', 'P>|z|']]).reset_index(drop=True)
coefficients4 = add_significance_stars(summary_model4[['Coef.', 'P>|z|']]).reset_index(drop=True)
coefficients5 = add_significance_stars(summary_model5[['Coef.', 'P>|z|']]).reset_index(drop=True)

# Combine the results from all five models side by side
combined_results = pd.concat([coefficients1[['Coef. (with stars)']],
                              coefficients2[['Coef. (with stars)']],
                              coefficients3[['Coef. (with stars)']],
                              coefficients4[['Coef. (with stars)']],
                              coefficients5[['Coef. (with stars)']]], axis=1)

# Rename the columns for clarity
combined_results.columns = ['Model 1 Coef.', 'Model 2 Coef.', 'Model 3 Coef.', 'Model 4 Coef.', 'Model 5 Coef.']

# Export to CSV or Excel for use in Word
combined_results.to_excel('../results/quasi_poisson_models_results.xlsx')

print("Model results with significance stars for five models exported successfully.")


In [ ]:
all_new_models = [baseline_qp_new, market_qp_new, governance_qp_new, statcap_qp_new, full_qp_new]

for model in all_new_models:
    print(check_overdispersion(model))


In [ ]:
# check if 'median_corrup_cases', 'median_audit_cases' are correlated. YES
# check if 'securityexp_pc', 'govexp_by_gdp' are correlated. NO
# check if 'landsale_by_govrev', 'median_land_size' are correlated. YES

In [ ]:
# correlation matrix
from scipy.stats import pearsonr

def calculate_pvalues(df):
    # Initialize an empty dataframe to store p-values
    pvalues = pd.DataFrame(np.ones((df.shape[1], df.shape[1])), columns=df.columns, index=df.columns)

    # Iterate over each pair of columns in the dataframe
    for col1 in df.columns:
        for col2 in df.columns:
            if col1 != col2:  # Avoid calculating p-values for diagonal
                corr, pval = pearsonr(df[col1], df[col2])
                pvalues.loc[col1, col2] = pval
    return pvalues

# Calculate the p-values for the correlation matrix
df_corr = df.drop(columns = ['province'])
pvalue_matrix = calculate_pvalues(df_corr)

# Select only the p-values less than 0.05
significant_pvalues = pvalue_matrix[pvalue_matrix < 0.05]

# Display the filtered p-values (NaN where p-values are >= 0.
print(significant_pvalues)

In [ ]:
df.info()

# Fitted values vs. true values

In [ ]:
import matplotlib.pyplot as plt

def plot_fitted_vs_observed(y, models, model_names, province_names):
    """
    Plots observed vs fitted values for multiple models in a 2-row, 3-column grid with province names.

    Parameters:
    - y: array-like, observed values.
    - models: list of models, each with a `.fittedvalues` attribute.
    - model_names: list of strings, names of the models.
    - province_names: list of strings, province names corresponding to the data points.

    Returns:
    - None
    """
    num_models = len(models)
    rows, cols = 2, 3  # Define grid layout
    
    # Create a figure with subplots
    fig, axes = plt.subplots(rows, cols, figsize=(15, 10))  # Adjust size dynamically
    axes = axes.flatten()  # Flatten the 2D array of axes for easy indexing

    for i, (model, model_name) in enumerate(zip(models, model_names)):
        fitted_values = model.fittedvalues  # Extract fitted values
        
        # Plot observed vs fitted values
        axes[i].scatter(y, fitted_values, label=f'{model_name}')
        axes[i].plot(y, y, '--', label='y = x')  # Reference line
        
        # # Annotate each point with its province name
        # for obs, fit, province in zip(y, fitted_values, province_names):
        #     axes[i].annotate(province, (obs, fit), fontsize=8, alpha=0.7)
        
        # Set plot details
        axes[i].set_title(f'{model_name}')
        axes[i].set_xlabel('Observed Values')
        axes[i].set_ylabel('Fitted Values')
        axes[i].legend()
        axes[i].grid(True)

    # Hide any unused subplots
    for j in range(num_models, len(axes)):
        axes[j].axis('off')  # Turn off axes for unused plots

    # Adjust layout for better spacing
    plt.tight_layout()

    # Show the figure
    plt.savefig('../results/fitted vs. observed plots.png')


# Assuming y is observed values, models is your list of models, and province_names is the list of province names
all_new_models = [baseline_qp_new, market_qp_new, governance_qp_new, statcap_qp_new, full_qp_new]
model_names = ['Model 0', 'Model 1', 'Model 2', 'Model 3', 'Model 4']
province_names = df['province'] # Replace with actual names

plot_fitted_vs_observed(y, all_new_models, model_names, province_names)


# Check specific variables

In [ ]:

# land sale by gov rev
plt.scatter(df['landsale_by_govrev'], y)

plt.ylabel("num of violence")
plt.xlabel("land sale to gov revenue")
plt.show()


In [ ]:
# land acquisition size
plt.scatter(df['median_land_size'], y)

# Add labels to each point
for i, label in enumerate(df['province']):  
    plt.text(df['median_land_size'][i], y[i], str(label), fontsize=9, ha='right')


plt.title('Land acquisition size vs. number of outsourced violence')
plt.ylabel("number of outsourced violence")
plt.xlabel("land acquisition size")
plt.savefig('../results/land_size_plot.png')
plt.show()

In [ ]:
# Calculate Pearson correlation coefficient and p-value
corr_coeff, p_value = pearsonr(df['land_size_square'], y)

print("Correlation Coefficient:", corr_coeff)
print("P-value:", p_value)

In [ ]:
outliers = ['Henan', 'Hebei', 'Shaanxi']
df_test = df[~df['province'].isin(outliers)]

In [ ]:
# Calculate Pearson correlation coefficient and p-value
corr_coeff, p_value = pearsonr(df_test['median_land_size'], df_test['median_corrup_cases'])

print("Correlation Coefficient:", corr_coeff)
print("P-value:", p_value)

In [ ]:
# corruption vs. audit

plt.scatter(df['median_audit_cases'], y)

# Add labels to each point
for i, label in enumerate(df['province']):  
    plt.text(df['median_audit_cases'][i], y[i], str(label), fontsize=9, ha='right')


plt.title('Audit cases vs. Anti-corruption cases')
plt.ylabel("Anti-corruption cases")
plt.xlabel("Audit cases")
# plt.savefig('../results/land_size_plot.png')
plt.show()

In [ ]:
# Calculate Pearson correlation coefficient and p-value
corr_coeff, p_value = pearsonr(df['median_audit_cases'], df['median_corrup_cases'])

print("Correlation Coefficient:", corr_coeff)
print("P-value:", p_value)

In [ ]:
df.info()

In [ ]:
# fiscal revenue vs. y

plt.scatter(df['govrev_by_gdp'], y)

# Add labels to each point
for i, label in enumerate(df['province']):  
    plt.text(df['govrev_by_gdp'][i], y[i], str(label), fontsize=9, ha='right')


plt.title('Fiscal revenue by gdp vs. number of violent cases')
plt.ylabel("# of violence")
plt.xlabel("Fiscal revenue by gdp")
plt.savefig('../results/fiscal_revenue_plot.png')
plt.show()

In [ ]:
# Calculate Pearson correlation coefficient and p-value
corr_coeff, p_value = pearsonr(df['reciprocal_govrev'], y)

print("Correlation Coefficient:", corr_coeff)
print("P-value:", p_value)

In [ ]:
df.columns

In [ ]:
x_list = ['gdp_pc', 'securityexp_pc', 'govrev_by_gdp', 'cpi', 'unemp_rate',
          'migpop_by_totalpop', 'median_audit_cases', 'median_corrup_cases', 'median_land_size']

# Create a single figure to hold all subplots
fig, axes = plt.subplots(len(x_list), 1, figsize=(10, len(x_list) * 5))

for i, x in enumerate(x_list):
    ax = axes[i]
    ax.scatter(df[x], y)

    # Add labels to each point
    for j, label in enumerate(df['province']):
        ax.text(df[x][j], y[j], str(label), fontsize=9, ha='right')

    # Set labels for axes
    ax.set_xlabel(x)
    ax.set_ylabel("# of violence")
    ax.set_title(f'Scatter plot of {x} vs. # of violence')

# Adjust layout
plt.tight_layout()

# Save all plots in one PNG file
plt.savefig('../results/all_scatter_plots.png')
